In [1]:
# Database schema plan for Microdose Social Platform

tables = {
    "users_social": [
        "id SERIAL PRIMARY KEY",
        "platform VARCHAR(50)",
        "platform_user_id VARCHAR(100) UNIQUE",
        "username VARCHAR(100)",
        "display_name VARCHAR(150)",
        "followers_count INT",
        "karma_score INT",
        "is_influencer BOOLEAN DEFAULT FALSE",
        "last_seen_at TIMESTAMP",
        "raw_user_json JSONB"
    ],
    "posts": [
        "id SERIAL PRIMARY KEY",
        "platform VARCHAR(50)",
        "platform_post_id VARCHAR(100) UNIQUE",
        "user_id INT REFERENCES users_social(id)",
        "url TEXT",
        "title TEXT",
        "text TEXT",
        "created_at_utc TIMESTAMP",
        "likes INT",
        "comments INT",
        "shares INT",
        "sentiment VARCHAR(20)",
        "topic_tags TEXT[]",
        "status VARCHAR(20)",
        "raw_post_json JSONB"
    ],
    "suggestions": [
        "id SERIAL PRIMARY KEY",
        "post_id INT REFERENCES posts(id)",
        "tone VARCHAR(50)",
        "text_original TEXT",
        "safety_flags TEXT[]",
        "created_at TIMESTAMP DEFAULT NOW()"
    ],
    "approvals": [
        "id SERIAL PRIMARY KEY",
        "suggestion_id INT REFERENCES suggestions(id)",
        "approver VARCHAR(100)",
        "action VARCHAR(20)",
        "edited_text TEXT",
        "timestamp TIMESTAMP DEFAULT NOW()"
    ],
    "publish_logs": [
        "id SERIAL PRIMARY KEY",
        "suggestion_id INT REFERENCES suggestions(id)",
        "platform VARCHAR(50)",
        "response_code INT",
        "response_body JSONB",
        "posted_at TIMESTAMP",
        "status VARCHAR(20)"
    ],
    "settings": [
        "id SERIAL PRIMARY KEY",
        "key VARCHAR(100) UNIQUE",
        "value JSONB",
        "updated_at TIMESTAMP DEFAULT NOW()"
    ]
}

for t, cols in tables.items():
    print(f"CREATE TABLE {t} (")
    print(",\n".join(cols))
    print(");\n")


CREATE TABLE users_social (
id SERIAL PRIMARY KEY,
platform VARCHAR(50),
platform_user_id VARCHAR(100) UNIQUE,
username VARCHAR(100),
display_name VARCHAR(150),
followers_count INT,
karma_score INT,
is_influencer BOOLEAN DEFAULT FALSE,
last_seen_at TIMESTAMP,
raw_user_json JSONB
);

CREATE TABLE posts (
id SERIAL PRIMARY KEY,
platform VARCHAR(50),
platform_post_id VARCHAR(100) UNIQUE,
user_id INT REFERENCES users_social(id),
url TEXT,
title TEXT,
text TEXT,
created_at_utc TIMESTAMP,
likes INT,
comments INT,
shares INT,
sentiment VARCHAR(20),
topic_tags TEXT[],
status VARCHAR(20),
raw_post_json JSONB
);

CREATE TABLE suggestions (
id SERIAL PRIMARY KEY,
post_id INT REFERENCES posts(id),
tone VARCHAR(50),
text_original TEXT,
safety_flags TEXT[],
created_at TIMESTAMP DEFAULT NOW()
);

CREATE TABLE approvals (
id SERIAL PRIMARY KEY,
suggestion_id INT REFERENCES suggestions(id),
approver VARCHAR(100),
action VARCHAR(20),
edited_text TEXT,
timestamp TIMESTAMP DEFAULT NOW()
);

CREATE TABLE p

In [1]:
# --- Influencer Thresholds (Default Values) ---
# These define the minimum follower/karma counts to classify a user as an influencer on each platform.

THRESHOLDS = {
    "twitter": 10_000,       # Twitter (X) users with >=10k followers
    "instagram": 10_000,     # Instagram users with >=10k followers
    "facebook": 10_000,      # Facebook pages/users with >=10k followers
    "reddit_karma": 10_000   # Reddit users with >=10k total karma
}


def is_influencer(platform: str,
                  followers_count: int | None = None,
                  karma_score: int | None = None,
                  thresholds: dict = THRESHOLDS) -> bool:
    """
    Check if a user qualifies as an influencer for a given platform.

    Args:
        platform (str): Social platform name (e.g., twitter, instagram, facebook, reddit)
        followers_count (int | None): Number of followers for the user (if applicable)
        karma_score (int | None): Reddit karma score (if applicable)
        thresholds (dict): Threshold dictionary for each platform

    Returns:
        bool: True if user meets influencer threshold, else False
    """

    p = platform.lower().strip()

    # For Reddit, influencer = karma >= threshold
    if p == "reddit":
        return (karma_score or 0) >= thresholds["reddit_karma"]

    # For Twitter/X, influencer = followers >= threshold
    elif p in ("twitter", "x"):
        return (followers_count or 0) >= thresholds["twitter"]

    # For Instagram, influencer = followers >= threshold
    elif p == "instagram":
        return (followers_count or 0) >= thresholds["instagram"]

    # For Facebook, influencer = followers >= threshold
    elif p == "facebook":
        return (followers_count or 0) >= thresholds["facebook"]

    # Default case: not an influencer
    return False


# --- Quick Test Section ---
# Testing sample users from different platforms to confirm logic works as intended

tests = [
    {"platform": "twitter", "followers": 12000, "karma": None},   # should be True
    {"platform": "instagram", "followers": 5400, "karma": None},  # should be False
    {"platform": "reddit", "followers": None, "karma": 11000},    # should be True
    {"platform": "facebook", "followers": 25000, "karma": None}   # should be True
]

# Print test results
for t in tests:
    print(
        t,
        "=>",
        is_influencer(t["platform"], t["followers"], t["karma"])
    )


{'platform': 'twitter', 'followers': 12000, 'karma': None} => True
{'platform': 'instagram', 'followers': 5400, 'karma': None} => False
{'platform': 'reddit', 'followers': None, 'karma': 11000} => True
{'platform': 'facebook', 'followers': 25000, 'karma': None} => True
